# GAIA Dataset Creator

we want to create a dataset for GAIA training

In [11]:
import json
import time
import pandas as pd
from huggingface_hub import InferenceClient
from llama_index import SimpleDirectoryReader
from llama_index.text_splitter import SentenceSplitter

In [10]:
documents = SimpleDirectoryReader("/home/leshkar/Desktop/allGaia/gaia/data").load_data()

In [ ]:
parser = SentenceSplitter(
    chunk_size=128,
    chunk_overlap=10,
    include_prev_next_rel=False,
)
nodes = parser.get_nodes_from_documents(documents)

In [ ]:
client = InferenceClient(model="http://gaia-u-01.westeurope.cloudapp.azure.com:8080")
max_qna = 10  
qna_count = 0 
qna_prompt = """I want you to help me create a question and an answer for training a large language model. 
    Use the provided context that contains the a paragraph of text and write a question and an answer on this context. 
    Don't write anything except the question and the answer. your reply should start with ###Human: 
    The format of your answer should be: 
    ### Human ###
    <question> 
    ### Assitant ###
    <answer> 
    """

In [16]:
SentenceSplitter??


Init signature:
SentenceSplitter(
    separator: str = ' ',
    chunk_size: int = 1024,
    chunk_overlap: int = 200,
    tokenizer: Optional[Callable] = None,
    paragraph_separator: str = '\n\n\n',
    chunking_tokenizer_fn: Optional[Callable[[str], List[str]]] = None,
    secondary_chunking_regex: str = '[^,.;。？！]+[,.;。？！]?',
    callback_manager: Optional[llama_index.callbacks.base.CallbackManager] = None,
    include_metadata: bool = True,
    include_prev_next_rel: bool = True,
    id_func: Optional[Callable[[int, llama_index.schema.Document], str]] = None,
) -> None
Source:        
class SentenceSplitter(MetadataAwareTextSplitter):
    """Parse text with a preference for complete sentences.

    In general, this class tries to keep sentences and paragraphs together. Therefore
    compared to the original TokenTextSplitter, there are less likely to be
    hanging sentences or parts of sentences at the end of the node chunk.
    """

    chunk_size: int = Field(
        default=D

In [ ]:
######### measure time #########
start_time = time.time() 
######### measure time #########

generated_qna = []
for node in nodes:
    node_text = node.get_content()  
    query = qna_prompt + f"\n\nContext: {node_text}"
    response = client.text_generation(prompt=query, max_new_tokens=1000)
    generated_text = response  
    split_text = generated_text.split("###Human:")  
    if len(split_text) > 1:
        answer_split = split_text[1].split("###Assistant:")
        if len(answer_split) > 1:
            question = answer_split[0].strip()  
            answer = answer_split[1].strip() 
        else:
            question = ""
            answer = ""
    else:
        question = ""
        answer = ""
    if question != "" and answer != "" and node_text != "":
        generated_qna.append({
            "context": node_text,
            "question": question,
            "answer": answer
        })
        qna_count += 1
        if qna_count >= max_qna:
            break 

######### measure time #########
end_time = time.time()
######### measure time #########
duration = end_time - start_time
print(f"Done! Total time taken: {duration:.2f} seconds, for {max_qna} qna sets.") 

In [ ]:
with open("generated_qna.json", "w") as json_file:
    json.dump(generated_qna, json_file)

In [14]:
data = pd.read_json('geenerated_qna.json')
data.to_parquet('output.parquet', index=False)

In [19]:
with open('geenerated_qna.json', 'r') as file:
    data = pd.read_json(file)
def transform_to_text(row):
    return f"<s>[INST] {row['context']} \n\n{row['question']} [/INST] {row['answer']} </s>"
data['text'] = data.apply(transform_to_text, axis=1)
text_data = data[['text']]
text_data.to_parquet('output.parquet', index=False)
